In [358]:
import numpy as np
import random
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True 

# Left 20%, Right 80%

suppose the probability distribution of getting reward is constant.

only two choise, left or right.

there would be no states but the learning change would be marked by time

the randomness of exploring instead of always choose the maximum expectation is modelled by sigmoid function, which makes such randomness constantly changing along time

# Todo: add in more policies

In [404]:
# some constants
obs_size = 2 # get 0 or 1 as reward
action_size = 2 #turn left or right
alpha = 0.2 #learning rate
beta = -6 # sharpness of sigmoid; this could be dynamic
gamma = 0
iteration = 80000
every_it_print=1000

# Env Policy
left = 0.2 # probablity of reward in left
right = 1 - left # probability of reward in right

In [389]:
# sigmoid function for nd; return the value of sigmoid with argument beta
# q can be n dimensional, either list or int/float
# beta should be between 0 and 1; determine the sharpness of sigmoid
# we can also try different functions here
def sigmoid(beta,q):
    q=np.array(q)
    val = 1/(1+np.exp(0-beta*q))
    return val

In [396]:
# Q table; 2x1


In [424]:
# Experiment
# reward = (1,0) represent reward is on the left, (0,1) right
# turn = (1,0) represent rat is on the left, (0,1) right

l_rat = 0.5 # evaluation from rat on how likely the reward is on left
r_rat = 0.5 # that for right
reward_rat = 0 # reward got by rat by last action; = reward
exp = 0 # action that maximize reward; 0 for left, 1 for right

count = 0 # count for continuous success of choosing the advantageous action(here, the right)
Q = np.zeros(2)

#logs
rewards=[]

for i in range(iteration):
            
    # 0. get choice from rat 
    ratio = l_rat/(l_rat+r_rat)
    if ratio > random.uniform(0,1):
        turn = np.array([1,0])
    else:
        turn = np.array([0,1])

    # 0. set reward 
    reward_site = random.uniform(0,1)
    if reward_site < right and np.array_equal(turn,np.array([0,1])):
    #if the rat goes right and the reward is on the right
        reward = np.array((0,1))
    elif reward_site >= right and np.array_equal(turn,np.array([1,0])):
        reward = np.array((1,0))
    else:
        reward=np.array((0,0))
    rewards.append(reward)
    
    # 2. update rat's information 
    Q = Q + alpha*turn*(reward + gamma*np.max(Q)-Q) # I think this is where it went wrong
    # you should only update the action taken

    l_rat = 1-(sigmoid(beta,Q[0])/(sigmoid(beta,Q[0])+sigmoid(beta,Q[1])))
    r_rat=1-l_rat
    # I changed this function
    # 3. update count
    if np.array_equal(turn,np.array([1,0])):
        count = 0
    elif np.array_equal(turn,np.array([0,1])):
        count = count + 1
        
        
    # Show ongoing results
    if i%every_it_print==0:
#         print('Q-table',Q)
#         print(l_rat)

        print(Q,reward)



    # end experiment if rat made more than 10 times right turn, which has prob < 0.00098 by purely random choice
#     if i == 280: #abs(r_rat-right < 0.01) # count > 10
#         print('The rat might has learned somthing. Iteration number:')
#         print(Q)

#         break

[0.2 0. ] [1 0]
[0.20565641 0.98615465] [0 1]
[0.05148447 0.7197463 ] [0 0]
[0.02729802 0.74234068] [0 1]
[0.15894718 0.90048771] [0 1]
[0.03550208 0.99435758] [0 1]
[0.21751051 0.70470239] [0 1]
[0.040686   0.88229396] [0 1]
[0.00548637 0.84096467] [0 1]
[0.02240687 0.99292153] [0 1]
[0.05270767 0.78643596] [0 1]
[0.02185115 0.62799068] [0 0]
[0.12613082 0.6497607 ] [0 1]
[0.30620393 0.56529257] [0 0]
[0.18031084 0.90820858] [0 1]
[0.20115   0.9112828] [0 1]
[0.2667841  0.95818164] [0 1]
[0.18973449 0.9261738 ] [0 1]
[0.08485982 0.9326607 ] [0 1]
[0.00920214 0.68092371] [0 1]
[0.08912699 0.75785371] [0 0]
[0.11851304 0.79892138] [0 0]
[0.1317944  0.94924216] [0 1]
[0.03275856 0.83070151] [0 1]
[0.02694255 0.835411  ] [0 1]
[0.14014325 0.72269458] [0 1]
[0.05281038 0.85188266] [0 1]
[0.16095135 0.72510061] [0 1]
[0.29468174 0.73639267] [0 1]
[0.16868588 0.56047704] [0 0]
[0.34869726 0.68766662] [0 1]
[0.1160436  0.88624872] [0 1]
[0.04873609 0.79425782] [0 1]
[0.0013718  0.76786173] [0

In [426]:
sum(rewards)[1]/sum(rewards)[0]

94.8498452012384

In [407]:
1-(sigmoid(beta,0.8)/(sigmoid(beta,0.8)+sigmoid(beta,0.2)))

0.9659378796930785

In [370]:
type(random.uniform(0,1))

float

In [247]:
env=[0.2,0.8]
train_size=5000
def get_reward(action,env): # 0 when reward is on the left
    if np.random.uniform(0,1)<env[1]:
        return int(action==1)
    else:
        return int(action==0)
def get_action(Q):
    return np.argmax(Q)

In [325]:
# second implementation
actions = np.array([0,1]) # 0 for left, 1 for right
Q = np.random.uniform(low=0,high=1,size=(action_size))
for i in range(train_size):
    action=get_action(Q)
    reward=get_reward(action,env)
    Q[action]=(1-alpha)*Q[action]+alpha*(reward+gamma*np.max(Q))
print(Q)

[2.37357591e-01 7.98636218e+02]


In [306]:
get_reward(1,env)

0

In [307]:
np.array_equal(test,np.array([1,2]))

True